In [ ]:
#Imports
import pandas as pd
import os
from google.cloud import bigquery
from datetime import datetime, date, timedelta
import re
import calendar
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from PIL import Image

%pip install Prophet 
from prophet import Prophet 

pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'

from pathlib import Path 

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\Greg\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\prophet\\stan_model\\cmdstan-2.33.1\\stan\\lib\\stan_math\\lib\\tbb_2020.3\\include\\tbb\\internal\\_deprecated_header_message_guard.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths




  Using cached prophet-1.1.6-py3-none-win_amd64.whl.metadata (3.6 kB)
  Using cached cmdstanpy-1.2.4-py3-none-any.whl.metadata (4.1 kB)
  Using cached holidays-0.59-py3-none-any.whl.metadata (25 kB)
  Using cached tqdm-4.66.6-py3-none-any.whl.metadata (57 kB)
  Using cached importlib_resources-6.4.5-py3-none-any.whl.metadata (4.0 kB)
  Using cached stanio-0.5.1-py3-none-any.whl.metadata (1.6 kB)
Using cached prophet-1.1.6-py3-none-win_amd64.whl (13.3 MB)
Using cached cmdstanpy-1.2.4-py3-none-any.whl (94 kB)
Using cached holidays-0.59-py3-none-any.whl (1.1 MB)
Using cached tqdm-4.66.6-py3-none-any.whl (78 kB)
Using cached importlib_resources-6.4.5-py3-none-any.whl (36 kB)
Using cached stanio-0.5.1-py3-none-any.whl (8.1 kB)


In [ ]:
#Bigquery credentials/Date transformations
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\Greg\OneDrive - Consumer Edge Research LLC\Desktop\Python\gcp-key-2.json"
#C:\Users\Greg\AppData\Local\Programs\Python\Python39\gcp-key-2.json
#C:\Users\Greg\OneDrive - Consumer Edge Research LLC\Desktop\Python\gcp-key-2.json
client = bigquery.Client()

#Trans_date transformations
def date_transf(df):
    df['trans_date_dt'] = pd.to_datetime(df['trans_date'], format = "%Y-%m-%d", errors = 'coerce')
    df['year'] = df['trans_date_dt'].dt.year.astype(str)
    df['month'] = df['trans_date_dt'].dt.month.apply(lambda x: calendar.month_abbr[x])
    df['norm_date'] = df['trans_date_dt'].dt.strftime("%m/%d/%Y")
    df['period'] = df['trans_date_dt'].dt.to_period('Q')
    df['norm_date_ya'] = df['norm_date'].shift(364)

In [ ]:
raw_hist_sql = 'SELECT distinct emax.segment_name, emax.brand_name, emax.trans_date, emax.cardtype, emax.spend_amount_usd, emax.trans_count FROM `beta-omega-alpha-theta.apollo_us_cube_daily_signal.day_sym_seg_brand_cardtype_emax` as emax LEFT JOIN `beta-omega-alpha-theta.ground_truth.symbol_detail` as sym_det ON emax.symbol_id = sym_det.symbol_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.industry_list` as ind ON sym_det.subindustry_id = ind.subindustry_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.symbol_segment` sym_seg ON emax.segment_name = sym_seg.segment_name LEFT JOIN `beta-omega-alpha-theta.ground_truth.segment_product` seg_prod ON sym_seg.segment_id = seg_prod.segment_id where seg_prod.primary_segment_flag = 1 and product_id = 1000 and emax.segment_name NOT LIKE "%ACQ%" and emax.segment_name NOT LIKE "%DLST%" and sym_det.ownership_status NOT LIKE "%Private%"'
raw_hist_df = client.query(raw_hist_sql).to_dataframe()